# Combine All the Batches

In [2]:
import pandas as pd
import os

In [ ]:
# replace with your folder's path
folder_path = r'/Users/judyliu/Desktop/csv files'

all_files = os.listdir(folder_path)

# Filter out non-CSV files
csv_files = [f for f in all_files if f.endswith('.csv')]

# Create a list to hold the dataframes
df_list = []

for csv in csv_files:
    file_path = os.path.join(folder_path, csv)
    try:
        # Try reading the file using default UTF-8 encoding
        df = pd.read_csv(file_path)
        df_list.append(df)
    except UnicodeDecodeError:
        try:
            # If UTF-8 fails, try reading the file using UTF-16 encoding with tab separator
            df = pd.read_csv(file_path, sep='\t', encoding='utf-16')
            df_list.append(df)
        except Exception as e:
            print(f"Could not read file {csv} because of error: {e}")
    except Exception as e:
        print(f"Could not read file {csv} because of error: {e}")

# Concatenate all data into one DataFrame
big_df = pd.concat(df_list, ignore_index=True)

# Drop the writers
big_df= big_df.drop(['writers'],axis=1)

# Save the final result to a new CSV file
big_df.to_csv(os.path.join(folder_path, 'combined_file.csv'), index=False)

# Import Data

In [ ]:
movie=pd.read_csv('/Users/judyliu/Desktop/csv files/combined_file.csv')

In [ ]:
movie.head()

,movie_id,directors,year,certificate,length,stars
0,42276,George Cukor,1950,Not Rated,1h 43m,Judy Holliday | William Holden | Broderick Cra...
1,86969,Alan Parker,1984,R,2h,Matthew Modine | Nicolas Cage | John Harkins
2,56875,Herschell Gordon Lewis,1963,Not Rated,1h 7m,William Kerwin | Mal Arnold | Connie Mason
3,62793,Winston Hibler,1967,Approved,1h 15m,Ron Brown | Brian Russell | Linda Wallace
4,86005,Carroll Ballard,1983,PG,1h 45m,Charles Martin Smith | Brian Dennehy | Zachary...


In [ ]:
movie.dtypes

movie_id        int64
directors      object
year           object
certificate    object
length         object
stars          object
dtype: object

In [ ]:
movie.shape

(9667, 6)

In [ ]:
print(movie.isnull().sum())

movie_id         0
directors        0
year             0
certificate      0
length         332
stars           94
dtype: int64


In [ ]:
movie['movie_id'] = movie['movie_id'].astype(str)
movie['movie_id'] = movie ['movie_id'].str.zfill(7)

# Extra Info Error Selection

In [ ]:
movie['year'].value_counts()

2002                          294
2001                          288
2006                          272
2014                          270
2000                          267
                             ... 
Episode aired Mar 15, 1991      1
Episode aired Mar 5, 2000       1
Episode aired May 23, 2005      1
1919                            1
Episode aired Sep 2, 2013       1
Name: year, Length: 151, dtype: int64

In [ ]:
movie_extra = (movie['year'].str.len()> 4)

In [ ]:
# Save the error movie_id into a file
extra = movie.loc[movie_extra].movie_id
extra.to_csv('error.csv', index=False)

In [ ]:
#Drop the error entries first
extra=extra.to_list()
movie.drop(movie[movie['movie_id'].isin(extra)].index, inplace=True)

# No Certificate Error

In [ ]:
movie.certificate.value_counts()

R            3211
PG-13        1744
PG           1510
Not Rated     795
G             298
             ... 
25m             1
1h              1
2h 6m           1
3h 11m          1
2h 29m          1
Name: certificate, Length: 149, dtype: int64

In [ ]:
# Save the movie_id of the nulls in length
movie_nc=movie[movie['length'].isnull()].movie_id.tolist()

#Assign 'length' to the right position
movie['length'] = movie['length'].fillna(movie['certificate'])

In [ ]:
#Replace 'unrated' in the certificate
movie.loc[movie['movie_id'].isin(movie_nc), 'certificate' ]= 'unrated'

In [ ]:
# Check one example
movie[movie['movie_id'] == '0110521']

,movie_id,directors,year,certificate,length,stars
3347,0110521,Martine Dugowson,1994,unrated,2h 8m,Romane Bohringer | Elsa Zylberstein | Florence...


# Combined extra_info clean data

In [ ]:
df2 = pd.read_csv('/Users/judyliu/Desktop/combine/combined.csv')

In [ ]:
movie = pd.concat([movie, df2], ignore_index=True)

In [ ]:
movie

,movie_id,directors,year,certificate,length,stars
0,0042276,George Cukor,1950,Not Rated,1h 43m,Judy Holliday | William Holden | Broderick Cra...
1,0086969,Alan Parker,1984,R,2h,Matthew Modine | Nicolas Cage | John Harkins
2,0056875,Herschell Gordon Lewis,1963,Not Rated,1h 7m,William Kerwin | Mal Arnold | Connie Mason
3,0062793,Winston Hibler,1967,Approved,1h 15m,Ron Brown | Brian Russell | Linda Wallace
4,0086005,Carroll Ballard,1983,PG,1h 45m,Charles Martin Smith | Brian Dennehy | Zachary...
...,...,...,...,...,...,...
9662,101254,Michael Apted,1991,unrated,2h 3m,Bruce Balden | Jacqueline Bassett | Symon Bast...
9663,103516,Stars | Helen Mirren | Colin Salmon | John Ben...,1992,unrated,3h 23m,Helen Mirren | Colin Salmon | John Benfield
9664,145600,Leslie Libman | Larry Williams,1998,unrated,1h 27m,Peter Gallagher | Leonard Nimoy | Tim Guinee
9665,233298,Curt Geda,2000,unrated,1h 16m,Will Friedle | Mark Hamill | Kevin Conroy


# Cluster Certificates

In [ ]:
movie.certificate.unique()

array(['Not Rated', 'R', 'Approved', 'PG', 'PG-13', 'Passed', 'TV-PG',
       'G', 'TV-G', 'unrated', '13+', 'GP', 'NC-17', 'X', 'M', 'Unrated',
       'M18', 'R(A)', 'NC-16', 'PG13', 'NC16', '(Banned)', 'TV-14', 'R21',
       'TV-MA', '15', 'M/PG', 'TV-Y7-FV', 'TV-13', '12+', '15+', 'R-18',
       'P', 'PG-12', '16+', '0+', '6+', '18+', 'R-12', 'AO', '7+', 'R-15',
       'U', 'TV-Y7', 'E'], dtype=object)

In [ ]:
rating_mapping = {
    'R': 'R',
    'M18': 'R',
    'R-18': 'R',
    'TV-MA': 'R',
    'R(A)': 'R',
    'NC-17': 'R',
    '18+': 'R',
    'X': 'R',
    'AO': 'R',
    'NC-16': '16+',
    'NC16': '16+',
    '16+': '16+',
    'M': '15+',
    'R-15': '15+',
    '15': '15+',
    '15+': '15+',
    'PG-13': 'PG13',
    'PG13': 'PG13',
    'TV-13': 'PG13',
    'R-12':'12+',
    'PG': 'PG',
    'TV-PG': 'PG',
    'GP': 'PG',
    'M/PG': 'PG',
    'Not Rated': 'unrated',
    'Unrated': 'unrated',
    '6+': '6+',
    'P': '6+',
    'TV-G': '6+',
    'TV-Y7-FV': '6+',
    'TV-Y7': '6+',
    'Approved': 'passed',
    'passed': 'passed'
}

movie = movie.replace(rating_mapping.keys(), rating_mapping.values())

In [ ]:
movie.certificate.unique()

array(['unrated', 'R', 'passed', 'PG', 'PG13', 'Passed', 'G', '6+', '13+',
       '15+', '16+', '(Banned)', 'TV-14', 'R21', '12+', 'PG-12', '0+',
       '7+', 'U', 'E'], dtype=object)

In [ ]:
print(movie.isnull().sum())

movie_id        0
directors       0
year            0
certificate     0
length          1
stars          94
dtype: int64


In [ ]:
#Drop null values
movie=movie.dropna()
movie

,movie_id,directors,year,certificate,length,stars
0,0042276,George Cukor,1950,unrated,1h 43m,Judy Holliday | William Holden | Broderick Cra...
1,0086969,Alan Parker,1984,R,2h,Matthew Modine | Nicolas Cage | John Harkins
2,0056875,Herschell Gordon Lewis,1963,unrated,1h 7m,William Kerwin | Mal Arnold | Connie Mason
3,0062793,Winston Hibler,1967,passed,1h 15m,Ron Brown | Brian Russell | Linda Wallace
4,0086005,Carroll Ballard,1983,PG,1h 45m,Charles Martin Smith | Brian Dennehy | Zachary...
...,...,...,...,...,...,...
9662,101254,Michael Apted,1991,unrated,2h 3m,Bruce Balden | Jacqueline Bassett | Symon Bast...
9663,103516,Stars | Helen Mirren | Colin Salmon | John Ben...,1992,unrated,3h 23m,Helen Mirren | Colin Salmon | John Benfield
9664,145600,Leslie Libman | Larry Williams,1998,unrated,1h 27m,Peter Gallagher | Leonard Nimoy | Tim Guinee
9665,233298,Curt Geda,2000,unrated,1h 16m,Will Friedle | Mark Hamill | Kevin Conroy


# Convert length to minutes

In [ ]:
def convert_to_minutes(length_str):
    if pd.isnull(length_str) or not isinstance(length_str, str):
        return None  # Return None for invalid inputs

    # Initialize hours and minutes to zero
    hours, minutes = 0, 0

    # Find hours and minutes in the string
    if 'h' in length_str:
        parts = length_str.split('h')
        hours = int(parts[0])
        length_str = parts[1] if len(parts) > 1 else ''
    if 'm' in length_str:
        minutes = int(length_str.replace('m', '').strip())

    # Calculate total minutes
    total_minutes = hours * 60 + minutes
    return total_minutes

In [ ]:
movie['length'] = movie['length'].apply(convert_to_minutes)

movie.rename(columns={'length': 'length (min)'}, inplace=True)

In [ ]:
movie

,movie_id,directors,year,certificate,length (min),stars
0,0042276,George Cukor,1950,unrated,103,Judy Holliday | William Holden | Broderick Cra...
1,0086969,Alan Parker,1984,R,120,Matthew Modine | Nicolas Cage | John Harkins
2,0056875,Herschell Gordon Lewis,1963,unrated,67,William Kerwin | Mal Arnold | Connie Mason
3,0062793,Winston Hibler,1967,passed,75,Ron Brown | Brian Russell | Linda Wallace
4,0086005,Carroll Ballard,1983,PG,105,Charles Martin Smith | Brian Dennehy | Zachary...
...,...,...,...,...,...,...
9662,101254,Michael Apted,1991,unrated,123,Bruce Balden | Jacqueline Bassett | Symon Bast...
9663,103516,Stars | Helen Mirren | Colin Salmon | John Ben...,1992,unrated,203,Helen Mirren | Colin Salmon | John Benfield
9664,145600,Leslie Libman | Larry Williams,1998,unrated,87,Peter Gallagher | Leonard Nimoy | Tim Guinee
9665,233298,Curt Geda,2000,unrated,76,Will Friedle | Mark Hamill | Kevin Conroy


# Split the actors and directors into a list of string

In [ ]:
# Splitting the 'Actors' column by | and expanding into separate columns
movie['stars'] = movie['stars'].str.split('|')
movie['directors']= movie['directors'].str.split('|')

In [ ]:
movie.head()

,movie_id,directors,year,certificate,length (min),stars
0,0042276,[George Cukor],1950,unrated,103,"[Judy Holliday , William Holden , Broderick ..."
1,0086969,[Alan Parker],1984,R,120,"[Matthew Modine , Nicolas Cage , John Harkins]"
2,0056875,[Herschell Gordon Lewis],1963,unrated,67,"[William Kerwin , Mal Arnold , Connie Mason]"
3,0062793,[Winston Hibler],1967,passed,75,"[Ron Brown , Brian Russell , Linda Wallace]"
4,0086005,[Carroll Ballard],1983,PG,105,"[Charles Martin Smith , Brian Dennehy , Zach..."


# Save the clean data

In [ ]:
movie.to_csv('movie_clean.csv', index=False)

In [ ]:
df1 = pd.read_csv('/Users/judyliu/Desktop/movie_clean.csv')
df2 = pd.read_csv('/Users/judyliu/Desktop/ml-latest-small/links.csv')

In [ ]:
merged_df = df1.merge(df2, left_on='movie_id', right_on='imdbId', how='left').drop(['tmdbId', 'imdbId'], axis=1)

In [ ]:
merged_df

,movie_id,directors,year,certificate,length (min),stars,movieId
0,42276,['George Cukor'],1950,unrated,103,"['Judy Holliday ', ' William Holden ', ' Brode...",3341
1,86969,['Alan Parker'],1984,R,120,"['Matthew Modine ', ' Nicolas Cage ', ' John H...",3342
2,56875,['Herschell Gordon Lewis'],1963,unrated,67,"['William Kerwin ', ' Mal Arnold ', ' Connie M...",3344
3,62793,['Winston Hibler'],1967,passed,75,"['Ron Brown ', ' Brian Russell ', ' Linda Wall...",3345
4,86005,['Carroll Ballard'],1983,PG,105,"['Charles Martin Smith ', ' Brian Dennehy ', '...",3347
...,...,...,...,...,...,...,...
9568,101254,['Michael Apted'],1991,unrated,123,"['Bruce Balden ', ' Jacqueline Bassett ', ' Sy...",26712
9569,103516,"['Stars ', ' Helen Mirren ', ' Colin Salmon ',...",1992,unrated,203,"['Helen Mirren ', ' Colin Salmon ', ' John Ben...",26761
9570,145600,"['Leslie Libman ', ' Larry Williams']",1998,unrated,87,"['Peter Gallagher ', ' Leonard Nimoy ', ' Tim ...",27074
9571,233298,['Curt Geda'],2000,unrated,76,"['Will Friedle ', ' Mark Hamill ', ' Kevin Con...",27311


In [ ]:

# Assuming df is your DataFrame and 'column_name' is the column you want to move to the front
column_name = 'movieId'

# Move the column to the front of the DataFrame
merged_df.insert(0, column_name, merged_df.pop(column_name))

In [ ]:
merged_df

,movieId,movie_id,directors,year,certificate,length (min),stars
0,3341,42276,['George Cukor'],1950,unrated,103,"['Judy Holliday ', ' William Holden ', ' Brode..."
1,3342,86969,['Alan Parker'],1984,R,120,"['Matthew Modine ', ' Nicolas Cage ', ' John H..."
2,3344,56875,['Herschell Gordon Lewis'],1963,unrated,67,"['William Kerwin ', ' Mal Arnold ', ' Connie M..."
3,3345,62793,['Winston Hibler'],1967,passed,75,"['Ron Brown ', ' Brian Russell ', ' Linda Wall..."
4,3347,86005,['Carroll Ballard'],1983,PG,105,"['Charles Martin Smith ', ' Brian Dennehy ', '..."
...,...,...,...,...,...,...,...
9568,26712,101254,['Michael Apted'],1991,unrated,123,"['Bruce Balden ', ' Jacqueline Bassett ', ' Sy..."
9569,26761,103516,"['Stars ', ' Helen Mirren ', ' Colin Salmon ',...",1992,unrated,203,"['Helen Mirren ', ' Colin Salmon ', ' John Ben..."
9570,27074,145600,"['Leslie Libman ', ' Larry Williams']",1998,unrated,87,"['Peter Gallagher ', ' Leonard Nimoy ', ' Tim ..."
9571,27311,233298,['Curt Geda'],2000,unrated,76,"['Will Friedle ', ' Mark Hamill ', ' Kevin Con..."


In [ ]:
merged_df.rename(columns={'movie_id': 'imdbId'}, inplace=True)

In [ ]:
merged_df

,movieId,imdbId,directors,year,certificate,length (min),stars
0,3341,42276,['George Cukor'],1950,unrated,103,"['Judy Holliday ', ' William Holden ', ' Brode..."
1,3342,86969,['Alan Parker'],1984,R,120,"['Matthew Modine ', ' Nicolas Cage ', ' John H..."
2,3344,56875,['Herschell Gordon Lewis'],1963,unrated,67,"['William Kerwin ', ' Mal Arnold ', ' Connie M..."
3,3345,62793,['Winston Hibler'],1967,passed,75,"['Ron Brown ', ' Brian Russell ', ' Linda Wall..."
4,3347,86005,['Carroll Ballard'],1983,PG,105,"['Charles Martin Smith ', ' Brian Dennehy ', '..."
...,...,...,...,...,...,...,...
9568,26712,101254,['Michael Apted'],1991,unrated,123,"['Bruce Balden ', ' Jacqueline Bassett ', ' Sy..."
9569,26761,103516,"['Stars ', ' Helen Mirren ', ' Colin Salmon ',...",1992,unrated,203,"['Helen Mirren ', ' Colin Salmon ', ' John Ben..."
9570,27074,145600,"['Leslie Libman ', ' Larry Williams']",1998,unrated,87,"['Peter Gallagher ', ' Leonard Nimoy ', ' Tim ..."
9571,27311,233298,['Curt Geda'],2000,unrated,76,"['Will Friedle ', ' Mark Hamill ', ' Kevin Con..."


In [ ]:
df3 = pd.read_csv('/Users/judyliu/Desktop/ml-latest-small/movies.csv')

In [ ]:
merged2 = merged_df.merge(df3, on='movieId',how='left')

In [ ]:
merged2

,movieId,imdbId,directors,year,certificate,length (min),stars,title,genres
0,3341,42276,['George Cukor'],1950,unrated,103,"['Judy Holliday ', ' William Holden ', ' Brode...",Born Yesterday (1950),Comedy
1,3342,86969,['Alan Parker'],1984,R,120,"['Matthew Modine ', ' Nicolas Cage ', ' John H...",Birdy (1984),Drama|War
2,3344,56875,['Herschell Gordon Lewis'],1963,unrated,67,"['William Kerwin ', ' Mal Arnold ', ' Connie M...",Blood Feast (1963),Horror
3,3345,62793,['Winston Hibler'],1967,passed,75,"['Ron Brown ', ' Brian Russell ', ' Linda Wall...","Charlie, the Lonesome Cougar (1967)",Adventure|Children
4,3347,86005,['Carroll Ballard'],1983,PG,105,"['Charles Martin Smith ', ' Brian Dennehy ', '...",Never Cry Wolf (1983),Adventure|Drama
...,...,...,...,...,...,...,...,...,...
9568,26712,101254,['Michael Apted'],1991,unrated,123,"['Bruce Balden ', ' Jacqueline Bassett ', ' Sy...",35 Up (1991),Documentary
9569,26761,103516,"['Stars ', ' Helen Mirren ', ' Colin Salmon ',...",1992,unrated,203,"['Helen Mirren ', ' Colin Salmon ', ' John Ben...",Prime Suspect 2 (1992),Crime|Drama|Mystery|Romance|Thriller
9570,27074,145600,"['Leslie Libman ', ' Larry Williams']",1998,unrated,87,"['Peter Gallagher ', ' Leonard Nimoy ', ' Tim ...",Brave New World (1998),Sci-Fi
9571,27311,233298,['Curt Geda'],2000,unrated,76,"['Will Friedle ', ' Mark Hamill ', ' Kevin Con...",Batman Beyond: Return of the Joker (2000),Action|Animation|Crime|Sci-Fi|Thriller


In [ ]:

# Assuming df is your DataFrame and 'column_name' is the column you want to move to the front
column_0 = 'title'

# Move the column to the front of the DataFrame
merged2.insert(2, column_0, merged2.pop(column_0))

In [ ]:
merged2

,movieId,imdbId,title,directors,year,certificate,length (min),stars,genres
0,3341,42276,Born Yesterday (1950),['George Cukor'],1950,unrated,103,"['Judy Holliday ', ' William Holden ', ' Brode...",Comedy
1,3342,86969,Birdy (1984),['Alan Parker'],1984,R,120,"['Matthew Modine ', ' Nicolas Cage ', ' John H...",Drama|War
2,3344,56875,Blood Feast (1963),['Herschell Gordon Lewis'],1963,unrated,67,"['William Kerwin ', ' Mal Arnold ', ' Connie M...",Horror
3,3345,62793,"Charlie, the Lonesome Cougar (1967)",['Winston Hibler'],1967,passed,75,"['Ron Brown ', ' Brian Russell ', ' Linda Wall...",Adventure|Children
4,3347,86005,Never Cry Wolf (1983),['Carroll Ballard'],1983,PG,105,"['Charles Martin Smith ', ' Brian Dennehy ', '...",Adventure|Drama
...,...,...,...,...,...,...,...,...,...
9568,26712,101254,35 Up (1991),['Michael Apted'],1991,unrated,123,"['Bruce Balden ', ' Jacqueline Bassett ', ' Sy...",Documentary
9569,26761,103516,Prime Suspect 2 (1992),"['Stars ', ' Helen Mirren ', ' Colin Salmon ',...",1992,unrated,203,"['Helen Mirren ', ' Colin Salmon ', ' John Ben...",Crime|Drama|Mystery|Romance|Thriller
9570,27074,145600,Brave New World (1998),"['Leslie Libman ', ' Larry Williams']",1998,unrated,87,"['Peter Gallagher ', ' Leonard Nimoy ', ' Tim ...",Sci-Fi
9571,27311,233298,Batman Beyond: Return of the Joker (2000),['Curt Geda'],2000,unrated,76,"['Will Friedle ', ' Mark Hamill ', ' Kevin Con...",Action|Animation|Crime|Sci-Fi|Thriller


In [ ]:
merged2.to_csv('movie_clean.csv', index=False)

In [ ]:
df1 = pd.read_csv('/Users/judyliu/Desktop/movie_clean.csv')

# Clean up creators

In [25]:
movie_info_final = pd.read_csv('movie_info_final.csv')

In [26]:
#check directors
creators_directors_df = movie_info_final[movie_info_final['directors'].str.contains('Creators', na=False)]
creators_directors_df

,movieId,imdbId,title,year,certificate,length (min),stars,genres,directors,rating,rating_no (K)
9244,163809,3718778,Over the Garden Wall (2013),2014,PG,22,"['Elijah Wood ', ' Collin Dean ', ' Melanie Ly...",Adventure|Animation|Drama,"[' Katie Krentz ', 'Creators ', ' Patrick McHa...",8.8,69.0
9303,167570,4635282,The OA,2016–2019,R,60,"['Brit Marling ', ' Jason Isaacs ', ' Scott Wi...",(no genres listed),"['Creators ', ' Zal Batmanglij ', ' Brit Marli...",7.8,114.0
9367,171495,81846,Cosmos,1980,PG,780,"['Carl Sagan ', ' Jaromír Hanzlík ', ' Jonatha...",(no genres listed),"[' Carl Sagan ', 'Creators ', ' Steven Soter '...",9.3,43.0


In [27]:
# Remove 'Creators' in 'director' column
def clean_directors(s):
    # Directly remove "Creators" and extra spaces
    cleaned_str = s.replace('Creators', '').strip()
    try:
        items = ast.literal_eval(cleaned_str)
        if isinstance(items, list):
            cleaned_items = [item.strip() for item in items if item.strip()]
            return ', '.join(cleaned_items)
        else:
            return cleaned_str 
    except:
        return s

# Apply the cleaning function
movie_info_final['directors'] = movie_info_final['directors'].apply(clean_directors)

In [34]:
# Check the 'directors' column
movie_info_final_clean = pd.read_csv('movie_info_final_clean.csv')
creators_directors_df = movie_info_final_clean[movie_info_final_clean['directors'].str.contains('Creators', na=False)]
creators_directors_df

,movieId,imdbId,title,year,certificate,length (min),stars,genres,directors,rating,rating_no (K)


In [35]:
# Test
movie_directors_clean =movie_info_final_clean[movie_info_final_clean['movieId'] == 163809]
movie_directors_clean

,movieId,imdbId,title,year,certificate,length (min),stars,genres,directors,rating,rating_no (K)
9244,163809,3718778,Over the Garden Wall (2013),2014,PG,22,"['Elijah Wood ', ' Collin Dean ', ' Melanie Ly...",Adventure|Animation|Drama,"Katie Krentz, Patrick McHale",8.8,69.0
